In [23]:
import flywheel
import flywheel_gear_toolkit as gt
from pprint import pprint


In [24]:
fw = flywheel.Client()


In [25]:
user_info = fw.get_current_user()
print(
    f"Firstname: {user_info.firstname} \n"
    f"Lastname: {user_info.lastname} \n"
    f"Email: {user_info.email}"
)



Firstname: Xuchu 
Lastname: Liu 
Email: Xuchu_Liu@rush.edu


In [26]:
# import os
# import pandas as pd

# asher_20210301_path = "/mnt/Storage/Xuchu_Liu/Workspace/DataSet/FetalData/20210301_asher_normal"
# asher_20210301_df = pd.read_csv(os.path.join(asher_20210301_path, "Accession number lookup.csv"), sep=',', header=None, names=['idx', 'acc', 'pname', 'uuid'])
# for index, data in asher_20210301_df.iterrows():
#     full_path = os.path.join(asher_20210301_path, "exam_%06d" % (data.idx))
#     full_cmd = f"fw ingest dicom -d \"{full_path}\" fetal \"Matlab GUIs\" --de-identify --subject {data.pname} --session {data.acc} -y"
#     print(full_cmd)
#     !$full_cmd

# asher_20210819_path = "/mnt/Storage/Xuchu_Liu/Workspace/DataSet/FetalData/20210819_asher_normal"
# asher_20210819_df = pd.read_csv(os.path.join(asher_20210819_path, "Original acc vs new acc.csv"), sep=',', header=None, names=['idx', 'acc', 'pname'])
# for index, data in asher_20210819_df.iterrows():
#     full_path = os.path.join(asher_20210819_path, "exam_%06d" % (data.idx))
#     full_cmd = f"fw ingest dicom -d \"{full_path}\" fetal \"Matlab GUIs\" --de-identify --subject {data.pname} --session {data.acc} -y"
#     # print(full_cmd)
#     !$full_cmd


In [27]:
# project_sessions = fw.sessions()
# for session in project_sessions:
#     print(session.label)


In [28]:
my_session_label = "2579537"
my_session_id = fw.sessions.find(f"label=\"{my_session_label}\"")[0].id
print(my_session_id)


621512d7b5c1475bf2a3f7d1


In [29]:
my_session_id = '621512d7b5c1475bf2a3f7d1'
my_session = fw.get_session(my_session_id)
print(my_session.keys())
print(my_session.age)


dict_keys(['operator', 'label', 'info', 'project', 'uid', 'timestamp', 'timezone', 'subject', 'age', 'weight', '_id', 'info_exists', 'parents', 'created', 'modified', 'revision', 'permissions', 'group', 'project_has_template', 'satisfies_template', 'files', 'notes', 'tags', 'analyses'])
None


In [30]:
my_group = my_session.parents.group
print(my_group)


fetal


In [31]:
my_metadata = my_session.info
if "ohifViewer" not in my_metadata:
    my_metadata["ohifViewer"] = {}
ohif_metadata = my_metadata["ohifViewer"]
if "measurements" not in ohif_metadata:
    ohif_metadata["measurements"] = {}
measure_metadata = ohif_metadata["measurements"]
if "Length" not in measure_metadata:
    measure_metadata["Length"] = []
length_metadata = measure_metadata["Length"]



In [32]:
pprint(ohif_metadata["measurements"]["Length"])

[]


In [33]:
Length_Template2 = {
    "SOPInstanceUID": "1.3.6.1.4.1.9590.100.1.2.390791004711043483141411902812130522701",
    "SeriesInstanceUID": "1.3.6.1.4.1.9590.100.1.2.9637877712912484919030652463070943910",
    "StudyInstanceUID": "1.3.6.1.4.1.9590.100.1.2.12974746613731427721459401343723795747",
    "_id": "ee6a6643-51fa-4675-0026-7d09e7dcf873",
    "_measurementServiceId": "1702e07d-6f7c-ecd3-9be5-f53ef32c9f5d",
    "active": True,
    "color": "rgba(244,238,52,0.2)",
    "description": "manually",
    "flywheelOrigin": {"id": "Xuchu_Liu@rush.edu", "type": "user"},
    "frameIndex": 0,
    "handles": {
        "end": {
            "active": False,
            "highlight": True,
            "moving": False,
            "x": 253.75156838143033,
            "y": 228.69761606022584,
        },
        "start": {
            "active": False,
            "highlight": True,
            "x": 242.1882057716436,
            "y": 256.3212045169385,
        },
        "textBox": {
            "active": False,
            "allowedOutsideImage": True,
            "boundingBox": {"height": 25, "left": 595, "top": 343.5, "width": 82.15625},
            "drawnIndependently": True,
            "hasBoundingBox": True,
            "hasMoved": False,
            "movesIndependently": False,
            "x": 253.75156838143033,
            "y": 228.69761606022584,
        },
    },
    "imagePath": "1.3.6.1.4.1.9590.100.1.2.12974746613731427721459401343723795747$$$1.3.6.1.4.1.9590.100.1.2.9637877712912484919030652463070943910$$$1.3.6.1.4.1.9590.100.1.2.390791004711043483141411902812130522701$$$0",
    "invalidated": False,
    "length": 11.697728502691763,
    "lesionNamingNumber": 2,
    "location": "Brain",
    "measurementNumber": 2,
    "sliceNumber": 9,
    "timepointId": "TimepointId",
    "toolType": "Length",
    "userId": None,
    "uuid": "303c1a73-05f1-4098-ab23-409cc3ab2de8",
    "viewport": {
        "displayedArea": {
            "brhc": {"x": 512, "y": 512},
            "columnPixelSpacing": 0.390625,
            "presentationSizeMode": "NONE",
            "rowPixelSpacing": 0.390625,
            "tlhc": {"x": 1, "y": 1},
        },
        "hflip": False,
        "invert": False,
        "labelmap": False,
        "pixelReplication": False,
        "rotation": 0,
        "scale": 1.556640625,
        "translation": {"x": 0, "y": 0},
        "vflip": False,
        "voi": {"windowCenter": 471, "windowWidth": 936},
    },
    "visible": True,
}

In [34]:
Length_Template = {
    "active": False,
    "color": "rgba(255,255,255,0.2)",
    "description": "description",
    "flywheelOrigin": {"id": "userid", "type": "user"},
    "frameIndex": 0,
    "handles": {
        "end": {"active": False, "highlight": True, "moving": False, "x": 0, "y": 0,},
        "start": {"active": False, "highlight": True, "x": 0, "y": 0,},
        "textBox": {
            "active": False,
            "allowedOutsideImage": True,
            "drawnIndependently": True,
            "hasBoundingBox": False,
            "hasMoved": False,
            "movesIndependently": False,
            "x": 0,
            "y": 0,
        },
    },
    "imagePath": "1.3.6",
    "invalidated": False,
    "length": 0,
    "lesionNamingNumber": 1,
    "location": "Brain",
    "measurementNumber": 1,
    "sliceNumber": 0,
    "SeriesInstanceUID": "1.3.6",
    "SOPInstanceUID": "1.3.6",
    "StudyInstanceUID": "1.3.6",
    "toolType": "Length",
    "visible": True,
}

In [47]:
abnormalColor = (60, 60, 250)
abnormalColor[0]

60

### def  addPons(self, file_id):

In [35]:
Pons_Length = Length_Template.copy()
Pons_Length["description"] = "A-P Diameter of Pons"


In [36]:
num_Length = len(ohif_metadata["measurements"]["Length"])
Pons_Length["handles"]["end"]["x"] = 244
Pons_Length["handles"]["end"]["y"] = 257
Pons_Length["handles"]["start"]["x"] = 258
Pons_Length["handles"]["start"]["y"] = 228
Pons_Length["handles"]["textBox"]["x"] = 258
Pons_Length["handles"]["textBox"]["y"] = 228
Pons_Length["length"]=12.58
Pons_Length["lesionNamingNumber"] = num_Length + 1
Pons_Length["location"]="Brain"
Pons_Length["measurementNumber"] = num_Length + 1
Pons_Length["sliceNumber"] = 9 # InstanceNumber
Pons_Length["toolType"]="Length"

In [37]:
user = fw.get_current_user()
Pons_Length["flywheelOrigin"]["id"]  = user.id

In [38]:
test_file = fw.get_file('621512dcb5c1475bf2a3f7ff')

Pons_Length["SeriesInstanceUID"] = test_file["info"]["SeriesInstanceUID"]
Pons_Length["SOPInstanceUID"] = test_file["info"]["SOPInstanceUID"]
Pons_Length["StudyInstanceUID"] = test_file["info"]["StudyInstanceUID"]


In [39]:
path_delimiter = "$$$"
path_suffix = "0"

imagePath = (
    f'{Pons_Length["StudyInstanceUID"]}'
    f'{path_delimiter}'
    f'{Pons_Length["SeriesInstanceUID"]}'
    f'{path_delimiter}'
    f'{Pons_Length["SOPInstanceUID"]}'
    f'{path_delimiter}'
    f'{path_suffix}'
)
Pons_Length["imagePath"] = imagePath
print(imagePath)

1.3.6.1.4.1.9590.100.1.2.12974746613731427721459401343723795747$$$1.3.6.1.4.1.9590.100.1.2.9637877712912484919030652463070943910$$$1.3.6.1.4.1.9590.100.1.2.390791004711043483141411902812130522701$$$0


In [40]:
Vermis_Length = {}
Vermis_Length["description"] = "A-P Diameter of Vermis"


In [41]:
HVermis_Length = {}
HVermis_Length["description"] = "Height of Vermis"


In [42]:
Fronto_Length = {}
Fronto_Length["description"] = "Fronto-Occpitial"


In [43]:
ohif_metadata["measurements"]["Length"].append(Pons_Length)
# ohif_metadata['measurements']['Length'].append(Vermis_Length)
# ohif_metadata['measurements']['Length'].append(HVermis_Length)
# ohif_metadata['measurements']['Length'].append(Fronto_Length)
# ohif_metadata['measurements']['Length'] = [] # delete


### def update(self):

In [44]:
my_session.update_info(my_metadata)


### import_data.py

In [ ]:
# If the "User Origin" column is not present in the Dataframe, generate it using
# the user ID of the person running this gear (or logged into the flywheel client)
user = fw.get_current_user()
user_id = user.id
print(user_id)


In [ ]:
context = gt.GearToolkitContext()
context.config_from_args([])
destination = context.destination


In [ ]:
destination = fw.get(my_session_id)  # destination = my_session
group = fw.get_group(destination.parents.group)  #  group object
project = fw.get_project(destination.parents.project)  # project object



In [ ]:
group_name = group.id
print(group_name)
project_name = project.label
print(project_name)
my_subject_label = "exam_000006"
subject = project.subjects.find(f"label={my_subject_label}")
# print(subject)


In [ ]:
############################################################################
# STEP 1: Loop through subject/session combos and aggregate file matches  #
############################################################################
# We will first loop through and find any and all matches for each subject/session combination.
# Group by subject/session combos, to minimize loading.
my_file_name = "7 - SAG BrainT2 HASTE BH.dicom.zip"
my_acquisitions = my_session.acquisitions()
my_acquisition = {}
my_file = None
for acquisition in my_acquisitions:
    for file in acquisition.files[:1]:
        if file.name.find(f"{my_file_name}") == 0:
            my_file = file  # flywheel.FileEntry
            my_acquisition = acquisition
            break
my_acquisition_label = my_acquisition.label
print(my_file)


In [ ]:
test_file = fw.get_file('61f008e991188d92013e6c57')
print(test_file)

In [ ]:
############################################################################
# STEP 2: Loop through aggregated matches and ensure there is only one     #
# Match per row                                                            #
############################################################################

# Now that we have assembled all the possible matches for every index in the dataframe, go through and make sure
# There aren't duplicates.
path = f"{group_name}/{project_name}/{my_subject_label}/{my_session_label}/{my_acquisition_label}/{my_file_name}"


In [ ]:
# roi = cu.get_roi_from_row(series, match.file, ses)
# series = df.loc[index]
# Class Match:
#     file: flywheel.FileEntry = None
# parent = match.file.parent


### csv_utils.py

In [ ]:
# def get_roi_from_row(series, file, session):
#     """Generate the dictionaries from a pandas series to create the ROI in flywheel

#     Args:
#         series (pandas.Series): a row from a dataframe describing an ROI
#         file (flywheel.FileEntry): a flywheel file to attach the ROI to
#         session (flywheel.Session): the parent session of the file

#     Returns:
#         roi (ROI_Template.ROI): a custom ROI object

#     """
# # Get the handle from the row.  The handle contains the x/y bounds of the ROI
# handle = get_handle_from_row(series)
# stats = get_stats_from_row(series)



In [ ]:
# def get_handle_from_row(series):
#     """Generate the flywheel handle from the series.

#     The only truly "required" columns needed to create a handle from the series are:
#     X min
#     Y min
#     X max
#     Y max

#     Args:
#         series (pandas.Series): a row from a dataframe describing an ROI

#     Returns:
#         handle_dict (dict): a nested dictionary object mirroring the "handle" object
#             used for the flywheel ROI metadata

#     """
# start_dict = {
#     "active": false,
#     "highlight": True,
#     "x": 280,
#     "y": 260
# }
# end_dict = {
#     "active": false,
#     "highlight": True,
#     "moving": false,
#     "x": 260,
#     "y": 240
# }
